In [1]:
# Define Julia packages
using LinearAlgebra, Symbolics

In [2]:
# Define Python packages
#import numpy as np

### Linear transformations as matrices - Python example

First, I want to show an example of the linear transformations fact I discussed from [slides 8-13](https://docs.google.com/presentation/d/132NHu6poeLCB606X4BHWjUz9mnr90yY8n7XhCU3Fqb8/edit?usp=sharing). 

I am defining three matrices A, B, C and a vector, x. At first I am multiplying ABC which we know is a composition equivalent to writing (a(b(c(x))). After multiplying these three matrices I apply this matrix with the variable name, `composition` to the vector x. 

Then, I change the strategy and multiply the respective matrix (A, B, C) to each output of the previous composition. For example: I multiply C(x) first and assign this a new variable name. Then I multiply B by the new variable name to get *yet* another matrix with a new variable name. Lastly I multiply A by this new matrix. 

I've used an assert statement to confirm that the output of the two approaches is the same. The first approach, multiply all matrices then apply it to x is my preferred approach. I believe this is also the standard way to do it.

Because we established these approaches are the same I will use the first approach in the Jacobian examples. 

In [3]:
# Define 3 matrices
#TODO: Later convert these constants to random ints: np.random.randint(x, y)

# m = 3
# q = 5
# p = 5
# n = 4

# # Set a seed 
# np.random.seed(0)

# A = np.random.rand(m, q)
# B = np.random.rand(q, p)
# C = np.random.rand(p, n)
# x = np.random.rand(n, 1)

In [4]:
# composition = A @ B @ C

In [5]:
# composition @ x

In [6]:
# c_to_x = C @ x
# b_to_Cx = B @ c_to_x
# A_to_BCx = A @ b_to_Cx
# A_to_BCx

In [7]:
# assert (composition @ x).all() == A_to_BCx.all()

### Example of f'(x), output of chain rule, being a product of two Jacobians - Julia example

On slide 32 of my [Introduction to Matrix Calculus](https://docs.google.com/presentation/d/132NHu6poeLCB606X4BHWjUz9mnr90yY8n7XhCU3Fqb8/edit?usp=sharing) presentation I derive the chain rule on arbitrary vectory spaces. I think it's helpful to see an example of this numerically. The functions in my example below are simple enough that we can compute the derivatives by hand to get more intuition for what's happening if we need. 

Let's first talk about our desired input and output dimensions. $x$ is a vector in $R^{n}$, $h(x)$ is a function (matrix) in $R^{p}$ and $f(x) = g(h(x))$ maps from $R^{n}$ to $R^{m}$. 

In the following section I'll refer to our desired "output" dimension so let me clarify what this means in a 2x2 example. 

If I have the matrix A which is a 2 x 2 matrix:

\begin{pmatrix}
a & b\\
c & d
\end{pmatrix}

and a vector x which 2 x 1:

\begin{pmatrix} x \\ y \end{pmatrix}

The matrix-vector multiplication dimensions are 2 x 2 (m x n) @ (2 x 1) (n x 1). So, in the language I'll use below, n is the input dimension and m is the output dimension. I am calling n the input because if we think back to linear transformations as matrices where A(x) = b, x is obviously the input to the function A and its dimensions are n x 1. 

$f'(x)$ is in the output dimension $R^{m}$ so we know $f'(x)$ is an m x n matrix. We also know $h(x)$ is in $R^{p}$ and its derivative $h'(x)$ is also in $R^{p}$ so h'(x) is a p x n matrix. Here, again, p is the output dimension. Therefore it follows that if m x n is our output and $h'(x)$ is p x n then $g'(x)$ is an m x p matrix. 

Given our knowledge that these primed matrices are Jacobians we can say that $f'(x)$, a Jacobian output is the product of an m x p Jacobian and a p x n Jacobian. 

---

In the code below I am using Julia's [ForwardDiff package](https://juliadiff.org/ForwardDiff.jl/stable/#:~:text=ForwardDiff%20implements%20methods%20to%20take,mode%20automatic%20differentiation%20(AD). The first function `z(x)` looks like the following in vector-valued function notation:

\begin{pmatrix}
x_{1}^{2} + x_{2}\\
x_{1}x_{2}
\end{pmatrix}

and the second function `w(y)` is:

\begin{pmatrix}
sin_{y1} + y_{2}\\
y_{1}y_{2}
\end{pmatrix}

Then, after defining x (the point we'd like to take partial derivatives with respect to) and evaluating z(x) at this point, I begin taking Jacobians. The reader can refer to slide 27 in [Introduction to Matrix Calculus](https://docs.google.com/presentation/d/132NHu6poeLCB606X4BHWjUz9mnr90yY8n7XhCU3Fqb8/edit?usp=sharing) to recall how to do these by hand and confirm the answers. 

In [8]:
using ForwardDiff

In [9]:
function z(x)
    return [x[1]^2 + x[2]
        x[1] * x[2]]
end

z (generic function with 1 method)

In [10]:
function w(y)
    return [sin(y[1]) + y[2],
    y[1] * y[2]]
end

w (generic function with 1 method)

In [11]:
x = [1.0, 2.0]

2-element Vector{Float64}:
 1.0
 2.0

In [12]:
zx = z(x)

2-element Vector{Float64}:
 3.0
 2.0

In [13]:
Dhx = ForwardDiff.jacobian(z, x)

2×2 Matrix{Float64}:
 2.0  1.0
 2.0  1.0

In [14]:
Dghx = ForwardDiff.jacobian(w, zx)

2×2 Matrix{Float64}:
 -0.989992  1.0
  2.0       3.0

In [15]:
J_composed = Dghx * Dhx

2×2 Matrix{Float64}:
  0.020015  0.0100075
 10.0       5.0

### Symbolic Jacobians in Symbolic Kronecker Product notation - Julia

The goal of this section is to develop a notation to write the Jacobian without having to write the Jacobian. This notation is called the Kronecker product. 

You may be asking: "Why would we want to do this?" It seems there are two reasons for this. First, it's simpler. As we will see we can write the Jacobian as a sum of two Kronecker products without needing to think about partial derivatives at all. Second, the Jacobian is sometimes computationally inefficient (the Kronecker product is too) and we'll want to adopt a different way to compute this. 

In order to understand the Kronecker product we will need to first understand matrix "vectorization." Let's also define and see an example of the Kronecker product. 

The first code block below defines a 2 x 2 matrix X. These dimensions (m x n) are important because the dimensions of our vectorized matrix will be mn x 1. Vectorizing a matrix stacks the columns on top of each other.

Next, let's define the Kronecker product. 

> If A is an m x n matrix with entries $a_{ij}$ and B is a p x q matrix with entries $b_{ij}$ then their Kronecker product A ⊗ B is an mp x nq matrix. The Kronecker product takes each entry of A and multiplies it by the matrix B s.t the resultant matrix has "all combinations of A's entries with the matrix B."

The example in the fourth code block below shows the definition clearly. The $X_{11}$ entry, **a** is multiplied by every letter in Z and this populates a mini 2x2 in `kron(X, Z)`. Perhaps one way to think about the Kronecker product, using the definition, is as a scalar multiplication of each $A_{ij}$ entry with B s.t. the result is an appropriately sized matrix in the Kronecker product output. The numerical example `kron(T, U)` shows this scalar multiplication perspective. 

Slides 45 - 49 of my [Introduction to Matrix Calculus](https://docs.google.com/presentation/d/132NHu6poeLCB606X4BHWjUz9mnr90yY8n7XhCU3Fqb8/edit?usp=sharing) presentation shares the linear operator perspective of the derivative of $A^{2}$ and I have included a dX variable and multiplied the Jacobian by dX and computed the X * dX + dX X form so you can see these are the same. 

In slides 52 through the end of the presentation we connect Jacobians to Kronecker Products. In the code blocks below, especially for the $d(A^{3})$ you can see the Jacobian and Kronecker Product giving the same answer.  

In [16]:
# Define variables
@variables a, b, c, d

X = [a b; c d]

2×2 Matrix{Num}:
 a  b
 c  d

In [17]:
# Vec the 2x2 matrix -> 4 x 1 vector

vec(X)

4-element Vector{Num}:
 a
 c
 b
 d

In [18]:
# First symbolic example of Kronecker products
@variables l, m, n, o

Z = [l m; n o]

2×2 Matrix{Num}:
 l  m
 n  o

In [19]:
kron(X, Z)

4×4 Matrix{Num}:
 a*l  a*m  b*l  b*m
 a*n  a*o  b*n  b*o
 c*l  c*m  d*l  d*m
 c*n  c*o  d*n  d*o

In [20]:
T = [1 2; 3 6]

2×2 Matrix{Int64}:
 1  2
 3  6

In [21]:
U = [10 20; 30 40]

2×2 Matrix{Int64}:
 10  20
 30  40

In [22]:
kron(T, U)

4×4 Matrix{Int64}:
 10   20   20   40
 30   40   60   80
 30   60   60  120
 90  120  180  240

In [23]:
X^2

2×2 Matrix{Num}:
 a^2 + b*c  a*b + b*d
 a*c + c*d  b*c + d^2

In [24]:
# Take the Jacobian of Y, vector valued function, with respect to parameters in X?

jac(Y, X) = Symbolics.jacobian(vec(Y), vec(X))

jac (generic function with 1 method)

In [25]:
# I think we would get the same answer if we took the Jacobian by hand. Right, the partial derivatives...
# ... of each term in X^2 are in the first row. 1,1 entry of X^2 is the first row of J
# ... and 2,1 entry of X^2 is the second row of J, etc. 

J = jac(X^2, X)

4×4 Matrix{Num}:
 2a      b      c   0
  c  a + d      0   c
  b      0  a + d   b
  0      b      c  2d

In [26]:
@variables dp, dq, dr, ds

dX = [dp dr; dq ds]
vec(dX)

4-element Vector{Num}:
 dp
 dq
 dr
 ds

In [27]:
vec(dX * X + X * dX)

4-element Vector{Num}:
       2a*dp + b*dq + c*dr
 a*dq + c*dp + c*ds + d*dq
 a*dr + b*dp + b*ds + d*dr
       b*dq + c*dr + 2d*ds

In [28]:
J * vec(dX)

4-element Vector{Num}:
      2a*dp + b*dq + c*dr
 (a + d)*dq + c*dp + c*ds
 (a + d)*dr + b*dp + b*ds
      b*dq + c*dr + 2d*ds

In [29]:
begin 
    I2 = [1 0; 0 1]
    K = kron(I2,X) + kron(X', I2)
end

4×4 Matrix{Num}:
 2a      b      c   0
  c  a + d      0   c
  b      0  a + d   b
  0      b      c  2d

In [30]:
jac(Y, X) = Symbolics.jacobian(vec(Y), vec(X))

jac (generic function with 1 method)

In [31]:
# Jacobian of X^3
J_2 = jac(X^3, X)

4×4 Matrix{Num}:
 3(a^2) + 2b*c         a*b + (a + d)*b              2a*c + c*d            b*c
    2a*c + c*d  a^2 + (a + d)*d + 2b*c                     c^2     a*c + 2c*d
    2a*b + b*d                     b^2  a*(a + d) + 2b*c + d^2     a*b + 2b*d
           b*c              a*b + 2b*d         (a + d)*c + c*d  2b*c + 3(d^2)

In [32]:
X2 = X^2

2×2 Matrix{Num}:
 a^2 + b*c  a*b + b*d
 a*c + c*d  b*c + d^2

In [33]:
begin
    kron(I2, X2) + kron(X', X) + kron(X2', I2)
end

4×4 Matrix{Num}:
 3(a^2) + 2b*c              2a*b + b*d              2a*c + c*d            b*c
    2a*c + c*d  a^2 + a*d + 2b*c + d^2                     c^2     a*c + 2c*d
    2a*b + b*d                     b^2  a^2 + a*d + 2b*c + d^2     a*b + 2b*d
           b*c              a*b + 2b*d              a*c + 2c*d  2b*c + 3(d^2)

In [34]:
# Convince myself that the vec() operator is necessary for the notation on slide 71 and elsewhere
vec(X*dX*I2 + I2*dX*X)

4-element Vector{Num}:
       2a*dp + b*dq + c*dr
 a*dq + c*dp + c*ds + d*dq
 a*dr + b*dp + b*ds + d*dr
       b*dq + c*dr + 2d*ds

In [35]:
K * vec(dX)

4-element Vector{Num}:
      2a*dp + b*dq + c*dr
 (a + d)*dq + c*dp + c*ds
 (a + d)*dr + b*dp + b*ds
      b*dq + c*dr + 2d*ds